In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from find_duplicates import find_duplicates, dataset2sketches

In [ ]:
import os

In [ ]:
os.path.split('./dataset/dump_part004.gz')

In [ ]:
with gzip.open('./dataset/dump_part004.gz', 'rb') as f:
    buffer = cStringIO.StringIO(f.read())
    reader = DocumentStreamReader(buffer)

In [ ]:
from os import listdir
from os.path import isfile, join
mypath = "./dataset/"

onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
onlyfiles

In [ ]:
import time

In [ ]:
times = []
for f in onlyfiles:
    s = time.time()
    dataset2sketches('./dataset/%s' % f)
    t = time.time() - s
    print t, f
    times.append(t)

In [ ]:
from find_duplicates import find_duplicates

In [ ]:
find_duplicates()

In [ ]:
from os import listdir
from os.path import isfile, join
mypath = "./csvs/"

csvs_files = [f for f in listdir(mypath) if isfile(join(mypath, f))]
csvs_files

In [ ]:
df = pd.concat([pd.read_csv("./csvs/%s" % f, index_col=0) for f in csvs_files])


In [ ]:
s = time.time()
ids = find_duplicates(df)
print time.time() - s

In [ ]:
from find_duplicates import list2comb

In [ ]:
s = time.time()
ids_pairs = ids.apply(list2comb).to
print time.time() - s

In [ ]:
all_pairs = reduce(lambda res, x: res + x, ids_pairs, [])

In [ ]:
df.shape[0]

In [ ]:
W = 5
N= 20

In [ ]:
target_df

In [ ]:
target_df.to_csv("result.csv")

In [ ]:
import mmh3
from collections import defaultdict, Counter

import numpy as np
from src.docreader import DocumentStreamReader
from src.htmlparse import parse_html

import gzip
import cStringIO

import pandas as pd

import itertools


In [ ]:
def words2shingles(words, W=5):
    return set([" ".join(words[i:i + W]) for i in range(0, len(words) - W)])

def minhashing(items, N=20):
    dictionary = defaultdict(list)
    for item in items:
        h = mmh3.hash(item)
        dictionary[h % N].append(h)
    minihashes = {k:sorted(set(v)) for k,v in dictionary.items()}
    return [v[0] for k,v in minihashes.items() if len(v) > 0]    

def doc2scratch(doc):
    words = parse_html(doc.body).encode("utf8").split(" ")
    shingles = words2shingles(words)
    
    sketch = minhashing(shingles)
    
    return pd.DataFrame(zip([doc.url]*len(sketch), sketch), columns=["id", "sketch"])

In [ ]:
with gzip.open('./dataset/dump_part004.gz', 'rb') as f:
    buffer = cStringIO.StringIO(f.read())
    reader = DocumentStreamReader(buffer)

In [ ]:
df = pd.DataFrame([])
for i, doc in enumerate(reader):
    scratch_df = doc2scratch(doc)
    df = pd.concat([df, scratch_df], ignore_index=True)

In [ ]:
df.shape

In [ ]:
def list2comb(x):
    return list(itertools.combinations(sorted(x), 2))

def find_duplicates(data_df):
    grouped = df.groupby(["sketch"])
    common = grouped.aggregate(lambda x: set(x))
    common["size"] = common["id"].apply(lambda x: len(x))
    ids = common[common["size"] > 1]["id"]
    
    ids_pairs = ids.apply(list2comb).tolist()
    
    all_pairs = reduce(lambda res, x: res+x, ids_pairs, [])
    counter = Counter(all_pairs)
    pairs, counts = zip(*counter.most_common())
    counts = np.array(counts)
    ratio = counts/float(np.max(counts))
    print np.max(counts)
    print ratio[:100]
    for p in pairs[:100]:
        print p[0], p[1]
#     print len(all_pairs)

In [ ]:
find_duplicates(df)